# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [4]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [7]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [8]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [9]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

## Матрица рейтингов

In [10]:
from scipy.sparse import coo_matrix, csr_matrix

In [11]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [12]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [13]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [14]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [15]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [16]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [17]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [18]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [19]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [20]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [21]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [22]:
from scipy.spatial.distance import cosine

def cosine_similarity_pair_users(u, v):
    rate_u = (u != 0)
    rate_v = (v != 0)
    sum_rates = rate_u.multiply(rate_v)
    form = (1 - cosine(u[sum_rates], v[sum_rates]))
    return 0 if sum_rates.nnz < 3 else form

In [23]:
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


Введите значение answer1 на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [34]:
from scipy.spatial.distance import cosine

def cosine_similarity_pair_users(u, v):
    idx = (u != 0).multiply(v != 0)
    return 0 if idx.nnz < 3 else (1 - cosine(u[idx], v[idx]))


def similar_users(u, R, n_neigbours):
    sim = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(sim)[::-1]
    return similar_users[:n_neigbours]

In [35]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [36]:
answer2

'42281633724815262065016921506'

Введите значение answer2 без кавычек  на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy. Это будет строка из 29 символов, которая начинается на 42.

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [76]:
def rate_items_user(u, R, n_neigbours):
    s_sum = 0.0
    predictions = csr_matrix((1, R.shape[1]))
    s = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(s)[::-1]
    for v in similar_users[1:n_neigbours]:
        predictions += s[v] * R[v]
        s_sum += np.abs(s[v])
    result_predict = predictions / s_sum
    return predictions / s_sum

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [102]:
rez = 2614, 306, 343, 5573, 6720
R_hat = rate_items_user(20, R, n_neigbours=30)
print(R_hat)

  (0, 0)	1.085995813128437
  (0, 1)	0.5859095674680375
  (0, 2)	0.10334154219685938
  (0, 6)	0.2068822994088008
  (0, 10)	0.15503679095540338
  (0, 16)	0.6203245132450556
  (0, 17)	0.12087230537310674
  (0, 18)	0.24127799397906274
  (0, 21)	0.15540724976542294
  (0, 24)	0.15511430464205403
  (0, 27)	0.2756080440419302
  (0, 30)	0.0862007609713551
  (0, 31)	0.862084262495876
  (0, 33)	0.24127616699844223
  (0, 35)	0.5344900168244411
  (0, 38)	0.10342604989761842
  (0, 46)	1.6896040304725384
  (0, 47)	0.22418783741130438
  (0, 49)	0.999821478169725
  (0, 55)	0.0861595289465024
  (0, 56)	0.2584415494024847
  (0, 58)	0.12074149964536056
  (0, 60)	0.13790119717990867
  (0, 61)	0.05173253982454854
  (0, 67)	0.13799151911219995
  :	:
  (0, 9478)	0.15503679095540338
  (0, 9490)	0.15540724976542294
  (0, 9496)	0.4656482518717203
  (0, 9523)	0.1550671977143027
  (0, 9530)	0.13783750907938017
  (0, 9531)	0.08624469944512499
  (0, 9538)	0.13792121755416814
  (0, 9571)	0.13783750907938017
  (0, 959

In [103]:
rated_items = (R[20] == 0)
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings = unseen_ratings.toarray()[0]
idx = unseen_ratings.argsort()[::-1]
unseen_ratings[idx]
top5 = idx[:5]
top5

array([2.34474168, 2.24147475, 2.15522858, ..., 0.        , 0.        ,
       0.        ])

array([2614,  306,  343, 5573, 6720], dtype=int64)

In [119]:
print(R_hat.toarray()[0][R_hat.toarray()[0].argsort()[::-1]])

[2.56903995 2.37968868 2.34474168 ... 0.         0.         0.        ]


## Генерация ответа

In [80]:
answer3 = ', '.join(str(i) for i in top5[:5])

In [81]:
answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy Формат ответа - строка вида "X, X, X, X, X", где X - идентификаторы. Вводить ответ следует без кавычек.